Deep Learning
=============

Assignment 5
------------

The goal of this assignment is to train a Word2Vec skip-gram model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
%matplotlib inline
from __future__ import print_function
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE

Download the data from the source website if necessary.

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


Read the data into a string.

In [3]:
def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words"""
  with zipfile.ZipFile(filename) as f:
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
  return data
  
words = read_data(filename)
print('Data size %d' % len(words))

Data size 17005207


Build the dictionary and replace rare words with UNK token.

In [4]:
vocabulary_size = 50000

def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count = unk_count + 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])
del words  # Hint to reduce memory.

Most common words (+UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156]


Function to generate a training batch for the skip-gram model.

In [5]:
data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [ skip_window ]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  return batch, labels

print('data:', [reverse_dictionary[di] for di in data[:8]])

for num_skips, skip_window in [(2, 1), (4, 2)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=8, num_skips=num_skips, skip_window=skip_window)
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])

data: ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first']

with num_skips = 2 and skip_window = 1:
    batch: ['originated', 'originated', 'as', 'as', 'a', 'a', 'term', 'term']
    labels: ['anarchism', 'as', 'a', 'originated', 'term', 'as', 'of', 'a']

with num_skips = 4 and skip_window = 2:
    batch: ['as', 'as', 'as', 'as', 'a', 'a', 'a', 'a']
    labels: ['a', 'originated', 'anarchism', 'term', 'of', 'as', 'term', 'originated']


Train a skip-gram model.

In [6]:
tf.reset_default_graph()

In [7]:
batch_size = 128
embedding_size = 128 # Dimension of the embedding vector.
skip_window = 1 # How many words to consider left and right.
num_skips = 2 # How many times to reuse an input to generate a label.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64 # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
  
  # Variables.
  embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  softmax_weights = tf.Variable(
    tf.truncated_normal([vocabulary_size, embedding_size],
                         stddev=1.0 / math.sqrt(embedding_size)))
  softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Model.
  # Look up embeddings for inputs.
  embed = tf.nn.embedding_lookup(embeddings, train_dataset)
  # Compute the softmax loss, using a sample of the negative labels each time.
  loss = tf.reduce_mean(
    tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, inputs=embed,
                               labels=train_labels, num_sampled=num_sampled, num_classes=vocabulary_size))

  # Optimizer.
  # Note: The optimizer will optimize the softmax_weights AND the embeddings.
  # This is because the embeddings are defined as a variable quantity and the
  # optimizer's `minimize` method will by default modify all variable quantities 
  # that contribute to the tensor it is passed.
  # See docs on `tf.train.Optimizer.minimize()` for more details.
  optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
  
  # Compute the similarity between minibatch examples and all embeddings.
  # We use the cosine distance:
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
    normalized_embeddings, valid_dataset)
  similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

In [ ]:
num_steps = 100001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  average_loss = 0
  for step in range(num_steps):
    batch_data, batch_labels = generate_batch(
      batch_size, num_skips, skip_window)
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    _, l = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += l
    if step % 2000 == 0:
      if step > 0:
        average_loss = average_loss / 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step %d: %f' % (step, average_loss))
      average_loss = 0
    # note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in range(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log = 'Nearest to %s:' % valid_word
        for k in range(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log = '%s %s,' % (log, close_word)
        print(log)
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step 0: 8.472012
Nearest to five: sound, irgun, modes, bracketed, trunk, detox, equalization, einsatzgruppen,
Nearest to world: violations, waff, origin, sof, pentafluoride, ting, indochina, darren,
Nearest to seven: infections, delirium, curran, operations, xxiv, albania, constructions, frigates,
Nearest to by: dimensions, centrioles, hemophilia, kusakabe, prevail, listed, sectarianism, provincia,
Nearest to s: nextel, mahoney, carangi, grows, hyrcanus, momentarily, dilmun, stadtholder,
Nearest to an: serra, packets, teeming, cram, breakdown, mathilde, lactic, glossary,
Nearest to have: mohawk, claude, comcast, nijmegen, abitibi, confessed, proclaim, cricket,
Nearest to however: sees, yao, leningrad, hamm, joachim, bernardo, relieving, aerosols,
Nearest to which: carbonates, sidonia, diplomas, peek, historicity, facilitates, dakar, carmilla,
Nearest to states: jewels, tirthankaras, gaulish, honolulu, publius, stores, had, susie,
Nearest to most: caucus, man

In [ ]:
num_points = 400

tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
two_d_embeddings = tsne.fit_transform(final_embeddings[1:num_points+1, :])

In [ ]:
def plot(embeddings, labels):
  assert embeddings.shape[0] >= len(labels), 'More labels than embeddings'
  pylab.figure(figsize=(15,15))  # in inches
  for i, label in enumerate(labels):
    x, y = embeddings[i,:]
    pylab.scatter(x, y)
    pylab.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points',
                   ha='right', va='bottom')
  pylab.show()

words = [reverse_dictionary[i] for i in range(1, num_points+1)]
plot(two_d_embeddings, words)

---

Problem
-------

An alternative to skip-gram is another Word2Vec model called [CBOW](http://arxiv.org/abs/1301.3781) (Continuous Bag of Words). In the CBOW model, instead of predicting a context word from a word vector, you predict a word from the sum of all the word vectors in its context. Implement and evaluate a CBOW model trained on the text8 dataset.

---

In [91]:
data_index = 0

In [92]:
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=[batch_size, 2 * skip_window], dtype=np.int32)
    labels = np.ndarray(shape=[batch_size, 1], dtype=np.int32)
    span = 2 * skip_window + 1
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window
        targets_to_avoid = []
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span-1)
            targets_to_avoid.append(target)
            batch_idx = list(range(span))[:target] + list(range(span))[target+1:]
            for k in range(len(batch_idx)):
                batch[i * num_skips + j, k] = buffer[batch_idx[k]]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels

In [94]:
for num_skips, skip_window in [(2, 1), (4, 2)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=8, num_skips=num_skips, skip_window=skip_window)
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    print('    batch:', [[reverse_dictionary[bi] for bi in batch[ki, :]] for ki in range(batch.shape[0])])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])


with num_skips = 2 and skip_window = 1:
    batch: [['anarchism', 'as'], ['anarchism', 'originated'], ['originated', 'a'], ['as', 'a'], ['as', 'term'], ['as', 'a'], ['a', 'of'], ['a', 'term']]
    labels: ['originated', 'as', 'as', 'originated', 'a', 'term', 'term', 'of']

with num_skips = 4 and skip_window = 2:
    batch: [['anarchism', 'originated', 'a', 'term'], ['originated', 'as', 'a', 'term'], ['anarchism', 'originated', 'as', 'term'], ['anarchism', 'originated', 'as', 'a'], ['originated', 'as', 'term', 'of'], ['originated', 'as', 'a', 'of'], ['originated', 'as', 'a', 'term'], ['originated', 'a', 'term', 'of']]
    labels: ['as', 'anarchism', 'a', 'term', 'a', 'term', 'of', 'as']


In [128]:
tf.reset_default_graph()

In [129]:
batch_size = 128
embed_dim = 512
num_skips = 2
skip_window = 2
num_sampled = 64
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))

with tf.name_scope('input'):
    train_data = tf.placeholder(tf.int32, shape=[None, 2 * skip_window], name='context')
    train_label = tf.placeholder(tf.int32, shape=[None, 1], name='label')
    valid_dataset = tf.constant(valid_examples, tf.int32, name='valid_data')
    
with tf.name_scope('word_embedding'):
    word_embed = tf.Variable(tf.random_uniform([vocabulary_size, embed_dim], -1, 1))
    embed = tf.nn.embedding_lookup(word_embed, train_data)

with tf.variable_scope('projection'):
    w1 = tf.get_variable('proj1', [embed_dim, embed_dim], initializer=tf.truncated_normal_initializer(
            stddev=1.0 / math.sqrt(embed_dim)))
    w2 = tf.get_variable('proj2', [embed_dim, embed_dim], initializer=tf.truncated_normal_initializer(
            stddev=1.0 / math.sqrt(embed_dim)))
    w3 = tf.get_variable('proj3', [embed_dim, embed_dim], initializer=tf.truncated_normal_initializer(
            stddev=1.0 / math.sqrt(embed_dim)))
    w4 = tf.get_variable('proj4', [embed_dim, embed_dim], initializer=tf.truncated_normal_initializer(
            stddev=1.0 / math.sqrt(embed_dim)))

with tf.name_scope('model'):
    p1 = tf.matmul(embed[:, 0], w1)
    p2 = tf.matmul(embed[:, 1], w2)
    p3 = tf.matmul(embed[:, 2], w3)
    p4 = tf.matmul(embed[:, 3], w4)
    p_sum = p1 + p2 + p3 + p4

with tf.name_scope('loss'):
    softmax_weight = tf.Variable(tf.truncated_normal([vocabulary_size, embed_dim],
                                 stddev=1.0 / math.sqrt(embed_dim)))
    softmax_bias = tf.Variable(tf.zeros([vocabulary_size]))
    loss = tf.reduce_mean(
        tf.nn.sampled_softmax_loss(weights=softmax_weight, biases=softmax_bias, inputs=p_sum,
                               labels=train_label, num_sampled=num_sampled, num_classes=vocabulary_size))

with tf.name_scope('optimizer'):
    optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
    
with tf.name_scope('similarity'):
    norm = tf.sqrt(tf.reduce_sum(tf.square(word_embed), 1, keep_dims=True))
    normalized_embeddings = word_embed / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

In [130]:
total_steps = 100001
step = 2000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    average_loss = 0
    for i in range(total_steps):
        batch_data, batch_label = generate_batch(batch_size, num_skips, skip_window)
        train_dict = {train_data: batch_data, train_label: batch_label}
        _, batch_loss = sess.run([optimizer, loss], feed_dict=train_dict)
        average_loss += batch_loss
        if (i+1) % step == 0:
            print('Loss: {:.4f}'.format(average_loss / step))
            average_loss = 0
        if (i+1) % 10000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8 # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k+1]
                log = 'Nearest to %s:' % valid_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log = '%s %s,' % (log, close_word)
                print(log)
    final_embeddings = normalized_embeddings.eval()

Loss: 2028.2289
Loss: 893.9122
Loss: 655.1231
Loss: 484.0254
Loss: 432.3917
Nearest to so: aristides, arches, palahniuk, touma, show, rockville, inherited, persecutions,
Nearest to one: two, btu, antonov, eight, joules, zero, three, nine,
Nearest to not: childbirth, successive, filament, willingly, abundances, been, figure, stresses,
Nearest to his: the, their, its, s, this, rimmed, fingerprinting, cyclist,
Nearest to is: are, was, were, became, as, has, of, crucifixes,
Nearest to no: decades, undergoes, televised, hark, adages, pessoa, tu, batter,
Nearest to which: nouveau, bassist, occam, precautions, baiting, meritorious, heterotic, who,
Nearest to than: amaziah, geckos, nucleon, type, taro, js, streets, opiates,
Nearest to are: is, were, was, have, be, miletus, islamists, has,
Nearest to only: correspondence, retention, morisot, printings, gently, concern, loss, inductee,
Nearest to during: meandering, un, mothers, from, slug, nonverbal, hometown, magnification,
Nearest to however: